In [2]:
import pandas as pd
import numpy as np


: 

In [3]:
N_ITERS = 1000 ## GIven in Question to run the algorithms for 1000 iterations

In [4]:
df = pd.read_csv('C:\\Users\\madha\\OneDrive\\Desktop\\bits\semII\\DRL\\assignment\\TrendMovie.csv') ## load dataset

In [5]:
df.head() ## checking rows

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
df['reward'] = (df['rating'] >= 4).astype(int) 
## Given in question:
# Reward=1
#  :Theuserratesthemoviehighstar(e.g., 4 or 5 stars).
#  ● 
# Reward=0
#  :Theuserratesthemovielowstar(e.g., 1, 2, or 3 stars)

In [12]:
df.head()

,userId,movieId,rating,timestamp,reward
0,1,1,4.0,964982703,1
1,1,3,4.0,964981247,1
2,1,6,4.0,964982224,1
3,1,47,5.0,964983815,1
4,1,50,5.0,964982931,1


In [7]:
n_movies = df['movieId'].nunique()
rewards = df.groupby('movieId')['reward'].mean()
rewards ##calc avg reward per movie id

movieId
1         0.683721
2         0.454545
3         0.346154
4         0.000000
5         0.244898
            ...   
193581    1.000000
193583    0.000000
193585    0.000000
193587    0.000000
193609    1.000000
Name: reward, Length: 9724, dtype: float64

### Random Policy

In [9]:
rewards_lst = []
for i in range(N_ITERS):
    movieid = np.random.choice(df['movieId'])
    reward = rewards[movieid]
    # print(i,reward)
    rewards_lst.append(reward)
   
random_policy_cum_rewards = np.cumsum(rewards_lst)  
print(random_policy_cum_rewards)

[  0.63636364   1.33317215   1.80614512   2.69503401   2.81268107
   2.81268107   2.81268107   3.42696678   3.90227542   4.35227542
   4.54582381   5.25342615   6.00342615   6.55550948   7.18735028
   7.68197394   8.07591333   8.35716333   8.99509436   9.49509436
  10.20938008  11.20938008  11.20938008  11.67279471  12.10136614
  12.5959898   13.40011351  14.01549812  14.01549812  14.68216479
  14.68216479  15.18216479  15.88258673  16.10480895  16.39052324
  16.80718991  17.55718991  18.01002009  18.46056954  19.07595416
  19.40928749  19.76113935  19.9739053   20.4114053   20.4114053
  20.9314053   21.65867803  22.06492803  22.71877418  22.96877418
  23.40995065  23.75995065  24.32574013  24.72574013  25.05907346
  25.33685124  26.13163951  26.28548567  26.75462147  26.92128814
  27.72128814  28.37346205  28.87346205  29.51694031  30.31846703
  30.76846703  31.26846703  31.51846703  31.68513369  31.68513369
  32.35180036  32.66430036  33.10874481  33.84207814  34.37064957
  34.846059

## Greedy Policy

In [10]:
def greedy_policy(iters):
    movie_estimated = {movie: 0 for movie in df['movieId'].unique()}
    movie_counts = {movie: 0 for movie in df['movieId'].unique()}
    rewards_lst = []

    for _ in range(iters):
        best_movie = max(movie_estimated, key=movie_estimated.get) ##selecting movie bsed on maximum reward value.
        reward = rewards[best_movie]
        rewards_lst.append(reward)
        
        movie_counts[best_movie] += 1
        movie_estimated[best_movie] += (reward - movie_estimated[best_movie]) / movie_counts[best_movie] 
        
    return np.cumsum(rewards_lst)

greedy_policy_cum_rewards = greedy_policy(N_ITERS)
print(greedy_policy_cum_rewards)

[  0.68372093   1.36744186   2.05116279   2.73488372   3.41860465
   4.10232558   4.78604651   5.46976744   6.15348837   6.8372093
   7.52093023   8.20465116   8.88837209   9.57209302  10.25581395
  10.93953488  11.62325581  12.30697674  12.99069767  13.6744186
  14.35813953  15.04186047  15.7255814   16.40930233  17.09302326
  17.77674419  18.46046512  19.14418605  19.82790698  20.51162791
  21.19534884  21.87906977  22.5627907   23.24651163  23.93023256
  24.61395349  25.29767442  25.98139535  26.66511628  27.34883721
  28.03255814  28.71627907  29.4         30.08372093  30.76744186
  31.45116279  32.13488372  32.81860465  33.50232558  34.18604651
  34.86976744  35.55348837  36.2372093   36.92093023  37.60465116
  38.28837209  38.97209302  39.65581395  40.33953488  41.02325581
  41.70697674  42.39069767  43.0744186   43.75813953  44.44186047
  45.1255814   45.80930233  46.49302326  47.17674419  47.86046512
  48.54418605  49.22790698  49.91162791  50.59534884  51.27906977
  51.9627907

### Epsilon

In [10]:
EPS_1 = 0.1
EPS_2 = 0.2
EPS_5 = 0.5

##values given in question

In [11]:
def epsilon_greedy_policy(iters, epsilon):
    estimated = {movie: 0 for movie in df['movieId'].unique()}
    counts = {movie: 0 for movie in df['movieId'].unique()}
    rewards_lst = []

    for _ in range(iters):
        if np.random.rand() < epsilon:  # Exploration
            movie_chosen = np.random.choice(df['movieId'])
        else:  ##exploitation
            movie_chosen = max(estimated, key=estimated.get)
        
        reward = rewards[movie_chosen]
        rewards_lst.append(reward)
        
        counts[movie_chosen] += 1
        estimated[movie_chosen] += (reward - estimated[movie_chosen]) / counts[movie_chosen]
    
    return np.cumsum(rewards_lst)

eps1_greedy_policy_cum_rewards = epsilon_greedy_policy(N_ITERS, EPS_1)
eps2_greedy_policy_cum_rewards = epsilon_greedy_policy(N_ITERS, EPS_2)
eps5_greedy_policy_cum_rewards = epsilon_greedy_policy(N_ITERS, EPS_5)

##running algorithm on given eps values.

In [ ]:
# UCB algorithm implementation
movie_counts = {movie_id: 0 for movie_id in df['movieId'].unique()}
movie_avg_rewards = {movie_id: 0 for movie_id in df['movieId'].unique()}
movie_total_rewards = {movie_id: 0 for movie_id in df['movieId'].unique()}

total_reward = 0

for i in range(1000):
    # Select the movie with the highest UCB
    ucb_values = {}
    for movie in movie_avg_rewards:
        if movie_counts[movie] > 0:
            ucb_values[movie] = movie_avg_rewards[movie] + np.sqrt(2 * np.log(i + 1) / movie_counts[movie])
        else:
            ucb_values[movie] = np.inf  # Choose untried movies in the beginning
    
    movie_chosen = max(ucb_values, key=ucb_values.get)
    
    
    # Update the statistics
    movie_counts[movie_chosen] += 1
    movie_avg_rewards[movie_chosen] = (movie_avg_rewards[movie_chosen] * (movie_counts[movie_chosen] - 1) + reward) / movie_counts[movie_chosen]    
    reward = rewards[movie_chosen]
    rewards_lst.append(reward)
    total_reward += reward
    
    # Print iteration info
    print(f"Iteration {i+1}: Movie {movie_chosen} recommended. Reward: {reward}")

print(f"Total reward after UCB: {total_reward}")

In [12]:
trace_random = go.Scatter(
    x=list(range(N_ITERS)),
    y=random_policy_cum_rewards,
    mode='lines',
    name='Random Policy'
)

trace_greedy = go.Scatter(
    x=list(range(N_ITERS)),
    y=greedy_policy_cum_rewards,
    mode='lines',
    name='Greedy Policy'
)

trace_eps_greedy_1 = go.Scatter(
    x=list(range(N_ITERS)),
    y=eps1_greedy_policy_cum_rewards,
    mode='lines',
    name='Epsilon Greedy Policy with eps 0.1'
)
trace_eps_greedy_2 = go.Scatter(
    x=list(range(N_ITERS)),
    y=eps2_greedy_policy_cum_rewards,
    mode='lines',
    name='Epsilon Greedy Policy with eps 0.2'
)

trace_eps_greedy_5 = go.Scatter(
    x=list(range(N_ITERS)),
    y=eps5_greedy_policy_cum_rewards,
    mode='lines',
    name='Epsilon Greedy Policy with eps 0.5'
)

fig = go.Figure(data=[trace_random,trace_greedy, trace_eps_greedy_1,trace_eps_greedy_2, trace_eps_greedy_5])

fig.update_layout(
    title="Cumulative Rewards Across Different Policies",
    xaxis_title="Number of Iterations",
    yaxis_title="Cumulative Reward",
    legend_title="Recommendation Policies",
    template="plotly_white",
    font=dict(size=14)
)

fig.show()
